In [20]:
import pygsheets
import pandas as pd
from tqdm import tqdm

In [2]:
client = pygsheets.authorize()

In [3]:
workers_ids = ['W1', 'W2', 'W3', 'W4', 'W5', 'W6']
id_to_key_task2_qa = {
    'ind': {
        'W1': '1LIVvmdGV6LHEZDxJjYVre9EkMwJ48tp6mpBPaz7LWoE',
        'W2': '1noJV_ViNfUGabWyWwGChEsVhfPc4ctxh2oVY9kcUaO4',
        'W3': '1M9QwUbbiHkZl_13Y-CNrfzNYN9No5hp_u6KWYhZdi1k',
        'W4': '1gz9l9wsY84cgtKJcYRCCuto7piBpv0DWY2jUDxsndI0',
        'W5': '1iPZV_yL3qwKmSi4mtDwhZNZlB90a63SoFRs2bChVeYc',
        'W6': '1YrprgKaeWU0L4jq78ioL40TyxEC0_nwGroEJVLqhjcg',
    },
    'sun': {
        'W1': '1MD7saLQYAXCLnpTA_WaV1oVv2vwbBn5hFfJ3z1s-Npg',
        'W2': '1Tn6nttAU3wAiMaJaBb6-P39IJ2sElMNS09bXUo-vI-Q',
        'W3': '1AQG9V74TxDkQAUt5aRs9mNa2fiyLMIML65X8nNQ-WuU',
        'W4': '1vSIJikiPedE4GNUB-zno2SN-L7c_e0IJMcrP8nq2RFk',
        'W5': '1b7G4c2PejzDpSqr4Nxo9sWYiAuE_egJS35Dt1HUkDvY',
        'W6': '1nu3ksVRHl-bTO0LW6yXIHGNhs7iJJ5AsdmHcrb2KxL0',
    }
}
id_to_key_task2 = {
    'ind': {
        'W1': '1Wa33qUjeB0pq1QI87jUqHTXlQ88ADCxAFp77vFAhSVU',
        'W2': '1FbYeTu3ZK4vBLoPVpJqCFB8Rj6yzazYRpxRtmPXXf-k',
        'W3': '1UysOeI1QnU8sNXqwQ7FGnULvKCjt8Tm6eoFEdYeQ0xY',
        'W4': '1lKIAWvs9D8JoyNrZJzB6pU4KVtfWstH3tOEfM9dZMDc',
        'W5': '1nQsQPAoxeRr9QzybjrkpTn5-IuBBCByMNN8txkUVV4w',
        'W6': '1SCV9OBxvHwxQ31t6GFirO6et-raMbMpJdfLjn1_HsgU',
    },
    'sun': {
        'W1': '1-FLTsgge53Wgb3HmIlM-oCOWow4_kLEJ0bey8MaMxVI',
        'W2': '1-WLiRHFXlD5BawHdBkI2kLSlLttBXU4ufnTwLB8pufM',
        'W3': '1EVv6ktg-6ZC5e9UBFvPIVrZI0T2WAbx3OHau_T-OgSE',
        'W4': '1XyXhn_R3VuNcsmHCmCEwojFM1hUPGhQe6FDrfo3ALds',
        'W5': '1YsreB2g0AeDbiFOIu2JOUmbAkwugBRaaf8SMUA2mrg0',
        'W6': '1oU4K52UaKJT3EEvDdOs8o4tD97Wxl4SKSFQK193w2bQ',
    }
}

In [27]:
lang = 'ind'

In [28]:
consistency_data = {}
for pred_id in tqdm(workers_ids):
    all_consts = []
    for ref_id in workers_ids:
        if pred_id == ref_id:
            all_consts.append(0)
        else:
            sh_pred = client.open_by_key(id_to_key_task2_qa[lang][pred_id])
            wks_pred = sh_pred.worksheet('title', ref_id)

            # Get inconsistent answers
            col_num  = 8 if lang == 'ind' else 9
            is_consistent = [a for a in wks_pred.get_col(col_num)[1:] if a != '']
            const, not_const = is_consistent.count('TRUE'), is_consistent.count('FALSE')
            if const + not_const != 250:
                print(pred_id, ref_id, const, not_const)
            assert const + not_const == 250
            all_consts.append(not_const)
    consistency_data[pred_id] = all_consts

100%|██████████| 6/6 [00:36<00:00,  6.16s/it]


In [29]:
consistency_data

{'W1': [0, 20, 13, 11, 26, 29],
 'W2': [13, 0, 13, 15, 15, 32],
 'W3': [11, 23, 0, 16, 33, 36],
 'W4': [11, 18, 15, 0, 31, 34],
 'W5': [18, 26, 23, 19, 0, 29],
 'W6': [18, 22, 23, 8, 26, 0]}

In [30]:
consistency_df = pd.DataFrame.from_dict(consistency_data, orient='index', columns=workers_ids)

In [31]:
consistency_df

,W1,W2,W3,W4,W5,W6
W1,0,20,13,11,26,29
W2,13,0,13,15,15,32
W3,11,23,0,16,33,36
W4,11,18,15,0,31,34
W5,18,26,23,19,0,29
W6,18,22,23,8,26,0
